# Notebook du micro-projet
## But du dataset:
Le fichier est disponible sur le git avec le nom "dataset.py" : [GitHub Project language_of_sign_recognition](https://github.com/morvan-s/language_of_sign_recognition)

Ce fichier récupère et prépare les données pour qu'elles soit exploité par le réseau de neurone.

## Déroulement du code :

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from keras.utils import np_utils, to_categorical
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import random

Commençons par bon nombre d'import. Principalement lié à Keras et Tensorflow (dont Keras est une sur-couche).

On va procéder a un formatage du jeu de données : on commence alors par aller chercher les différentes images dans images.npy. Ensuite on va assigner pour chaque image, le bon label (les labels fournis de base n'étant pas bien assigné). C'est ce qui va permettre d'entrainer le réseau et verifier qu'il a bien appris. Puis on va modifier le format des images avant de les passer dans le modele afin de faciliter tous les calculs/comparaison. Afin d'améliorer l'entrainement, on va mélanger le dataset, cela va permettre que le réseau ne traite pas toutes les valeurs de 9 puis toutes les valeurs de 0 etc. Cela améliorer les performances lors de l'entrainement, on fini en séparant en 80% des données pour l'entraînement et 20% pour le test.

In [2]:
def load_data():
    # Load images
    images = np.load('datasets/images.npy')
    classes = np.zeros(images.shape[0])
    classes[:204] = 9
    classes[204:409] = 0
    classes[409:615] = 7
    classes[615:822] = 6
    classes[822:1028] = 1
    classes[1028:1236] = 8
    classes[1236:1443] = 4
    classes[1443:1649] = 3
    classes[1649:1855] = 2
    classes[1855:] = 5
    
    classes = np_utils.to_categorical(classes, 10)
    images = images.reshape(images.shape[0], 64, 64, 1)
    
    combined = list(zip(images, classes))
    random.shuffle(combined)
    images[:], classes[:] = zip(*combined)
    
    splitIndex = int(len(images) * 0.8)
    x_train = images[:splitIndex]
    x_test = images[splitIndex:]
    y_train = classes[:splitIndex]
    y_test = classes[splitIndex:]
    
    return (x_train, y_train), (x_test, y_test)

## But du modèle :
Ce fichier est disponible sur le git avec le nom model.py : [GitHub Project language_of_sign_recognition](https://github.com/morvan-s/language_of_sign_recognition)

Ce fichier centralise une grande partie de se projet et en particulier le model qui va réunir les différentes couches du réseau.

## Déroulement du code :

Dans cette partie, on va comparer les résultats d'un modèle classique avec ceux d'un modèle convolutionnel. 

### Modèle classique :

In [9]:
(x_train, y_train), (x_test, y_test) = load_data()

model2 = Sequential()
model2.add(Flatten(input_shape=(64,64,1)))
model2.add(Dense(32, activation='relu'))
model2.add(Dense(16, activation='relu'))
model2.add(Dense(16, activation='relu'))
model2.add(Dense(10, activation='softmax'))

model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.fit(
        x_train,
        y_train,
        32,
        epochs=100,
        validation_data=(x_test, y_test)
)

Train on 1649 samples, validate on 413 samples
Epoch 1/100
1649/1649 [==============================] - 1s 413us/step - loss: 2.3149 - acc: 0.1195 - val_loss: 2.2151 - val_acc: 0.1162
Epoch 2/100
1649/1649 [==============================] - 0s 118us/step - loss: 2.2012 - acc: 0.1704 - val_loss: 2.0920 - val_acc: 0.2349
Epoch 3/100
1649/1649 [==============================] - 0s 113us/step - loss: 2.1173 - acc: 0.2292 - val_loss: 2.0198 - val_acc: 0.2833
Epoch 4/100
1649/1649 [==============================] - 0s 116us/step - loss: 2.0716 - acc: 0.2147 - val_loss: 1.9105 - val_acc: 0.4044
Epoch 5/100
1649/1649 [==============================] - 0s 101us/step - loss: 1.9018 - acc: 0.3384 - val_loss: 1.7328 - val_acc: 0.4455
Epoch 6/100
1649/1649 [==============================] - 0s 102us/step - loss: 1.8012 - acc: 0.3766 - val_loss: 1.9637 - val_acc: 0.2809
Epoch 7/100
1649/1649 [==============================] - 0s 110us/step - loss: 1.7845 - acc: 0.3602 - val_loss: 1.6791 - val_acc: 0

1649/1649 [==============================] - 0s 114us/step - loss: 0.6593 - acc: 0.7841 - val_loss: 1.0115 - val_acc: 0.6441
Epoch 61/100
1649/1649 [==============================] - 0s 112us/step - loss: 0.7367 - acc: 0.7477 - val_loss: 0.6784 - val_acc: 0.7700
Epoch 62/100
1649/1649 [==============================] - 0s 127us/step - loss: 0.6708 - acc: 0.7659 - val_loss: 0.7115 - val_acc: 0.7700
Epoch 63/100
1649/1649 [==============================] - 0s 112us/step - loss: 0.6895 - acc: 0.7544 - val_loss: 0.6586 - val_acc: 0.7651
Epoch 64/100
1649/1649 [==============================] - 0s 114us/step - loss: 0.6775 - acc: 0.7653 - val_loss: 0.8902 - val_acc: 0.6901
Epoch 65/100
1649/1649 [==============================] - 0s 109us/step - loss: 0.6546 - acc: 0.7659 - val_loss: 0.8232 - val_acc: 0.7191
Epoch 66/100
1649/1649 [==============================] - 0s 105us/step - loss: 0.6326 - acc: 0.7853 - val_loss: 0.5921 - val_acc: 0.8039
Epoch 67/100
1649/1649 [=======================

### Modèle convolutionnel :

In [34]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1), data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
model.add(Convolution2D(64, (2, 2), activation='relu', input_shape=(64, 64, 1), data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
model.add(Convolution2D(128, (2, 2), activation='relu', input_shape=(64, 64, 1), data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
model.add(Convolution2D(256, (2, 2), activation='relu', input_shape=(64, 64, 1), data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

**Commentaire sur le model**
On applique un modele convolutionnel classique: on alterne des couches de convolutions avec un filtre (2,2), une fonction d'activation Relu, et le data_format 'channels_last' qui est  à mettre en corrélation avec le formatage des données effectué à l'étape précéde,t. On alterne donc ces couches convolutionnelles avec de couches de max-pooling avec toujours un filtre (2,2) : cela permet de faire du sous-échantillonage et ainsi permettre au réseau d'extraire des features à différents niveau d'abstractions. On rajoute une couche flatten permettant de faire la liaisons entre les couches convolutionneles et les couches denses, les couches denses permettent d'obtenir le résultat final, la fonction softmax permet d'interpreter le résultat comme une probabilité. On a également ajouter deux couches de Dropout qui permettent de désactiver de manière aléaoire des sorties de neurones, cela nous as permis d'améliorer nos résultats.

Ensuite on compile ce model, pour ce faire on a 3 paramètres, en particulier l'optimizer (adam, optimiseur stochastique) et le loss (crossentropy car ce ne sont pas de simple entier) :

In [35]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

Il y a ensuite un load. Il appel une méthode de l'autre fichier python (DataSet).

In [36]:
(x_train, y_train), (x_test, y_test) = load_data()

Ici nous avons 4 paramètres d'initialiser :
- Epochs : Le nombre d'époque utiliser. Plus cette valeur est élever plus le réseau sera efficace. Mais plus le code sera long à  exécuter.
- Batch_size : La taille des différents lots.
- Data_augmentation : Comme son nom l'indique, ce booléen indique si on utilise la data augmentation (multiplié le nombre de données)

In [37]:
EPOCHS = 30
BATCH_SIZE = 32
DATA_AUGMENTATION = False

Ensuite, on va permettre l'augmentation de données, ce qui nous as permis de comparer les résultats avec/sans cette augmentation. L'augmentation de données fonctionne de la manière suivante : on va effectuer sur les images du dataset des transformations diverses (rotation, scaling...), cela permet d'avoir beaucoup plus de données et d'avoir un réseau de neurones qui apprendra a associer le bon label a une image par exemple un peu de travers ou autre. 
Nous avons réalisé une petite démonstration web à l'aide de TensorFlowJS, nous avons constaté de bien meilleur résultat avec l'augmentation de données.

In [41]:
if DATA_AUGMENTATION:
    # With data augmentation
    generator = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=40,
        width_shift_range=0.3,
        height_shift_range=0.3,
        zoom_range=[0.7, 1.3],
    )
    model.fit_generator(
        generator.flow(x_train, y_train, batch_size=BATCH_SIZE),
        steps_per_epoch=len(x_train) // BATCH_SIZE,
        epochs=EPOCHS
    )
else:
    # Without data augmentation
    model.fit(
        x_train,
        y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(x_test, y_test)
)

Train on 1649 samples, validate on 413 samples
Epoch 1/30
1649/1649 [==============================] - 12s 7ms/step - loss: 2.2011 - acc: 0.1389 - val_loss: 2.1080 - val_acc: 0.2373
Epoch 2/30
1649/1649 [==============================] - 11s 7ms/step - loss: 2.0430 - acc: 0.2583 - val_loss: 1.4449 - val_acc: 0.5109
Epoch 3/30
1649/1649 [==============================] - 11s 6ms/step - loss: 1.1844 - acc: 0.5785 - val_loss: 0.7600 - val_acc: 0.7579
Epoch 4/30
1649/1649 [==============================] - 10s 6ms/step - loss: 0.6834 - acc: 0.7599 - val_loss: 0.5378 - val_acc: 0.8475
Epoch 5/30
1649/1649 [==============================] - 10s 6ms/step - loss: 0.4907 - acc: 0.8351 - val_loss: 0.4034 - val_acc: 0.8886
Epoch 6/30
1649/1649 [==============================] - 9s 6ms/step - loss: 0.3821 - acc: 0.8678 - val_loss: 0.3658 - val_acc: 0.8886
Epoch 7/30
1649/1649 [==============================] - 9s 6ms/step - loss: 0.3326 - acc: 0.8848 - val_loss: 0.2777 - val_acc: 0.9274
Epoch 8/30

Le générateur est un ImageDataGenerator, générateur intégrer à Keras. Ici on a pris quelques décision pour l'augmentation tel que la rotation des images et l'intervalle de zoom. On utilise pour finir le model créer précédemment pour entraîné le réseau.
Pour finir on va tester l'efficacité du réseau de neurone grâce à la méthode evaluate du model.

In [42]:
loss, accuracy = model.evaluate(x_test, y_test, batch_size=128)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

413/413 [==============================] - 1s 2ms/step
Test loss: 0.04630225122011025
Test accuracy: 0.9903147699757869


## Démonstration :

Une démonstration en ligne a été faite pour ce projet (il suffit d'afficher le fichier index.html du dossier demo dans le navigateur).
Le gif ci-dessous montre le fonctionnement de cette démonstration.

![demonstration](demonstration.gif "demonstration")